In [1]:
import getpass
import pyspark
from pyspark.sql import SparkSession


spark = (SparkSession
         .builder
         .appName('final-proj')
         .master('yarn')
         .config('spark.port.maxRetries', 100)
         .config('spark.executor.memory', '1g')
         .config('spark.executor.instances', '2')
         .config('spark.executor.cores', '2')
         .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0')
         .getOrCreate())

sc = spark.sparkContext
conf = sc.getConf()

spark

In [2]:
# Metadata file contains all the stations and gps coordinates
metadata = spark.read.text('/datasets/project/metadata/BFKOORD_GEO')

In [3]:
# Splitting and adding new columns to the spark dataframe
split_col = pyspark.sql.functions.split(metadata['value'], " % ")
split_left = pyspark.sql.functions.split(split_col.getItem(0), " +")
metadata = metadata.withColumn('longitude', split_left.getItem(2))
metadata = metadata.withColumn('latitude', split_left.getItem(1))
metadata = metadata.withColumn('stop', split_col.getItem(1))
metadata = metadata.drop('value')

metadataPandas = metadata.toPandas()
metadataPandas.head()

,longitude,latitude,stop
0,44.446770,26.074412,Bucuresti
1,50.901549,1.811446,Calais
2,51.284212,1.075329,Canterbury
3,50.729172,-3.543547,Exeter
4,46.922368,9.733756,"Fideris, Bahnhof"


In [4]:
# Zürich HB is the starting point
zurich_coord = metadataPandas[metadataPandas['stop'] == 'Zürich HB']
zurich_coord

,longitude,latitude,stop
2379,47.378177,8.540192,Zürich HB


In [5]:
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance of gps coordinates
def compute_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0 
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance*1000


In [6]:
# keep only stations inside 10km
mask = metadataPandas.apply(lambda x: compute_distance(
    float(zurich_coord['latitude']), 
    float(zurich_coord['longitude']),
    float(x['latitude']), 
    float(x['longitude'])) <= 10000, axis=1)
metadataPandas = metadataPandas[mask]

In [7]:
df = spark.read.option("delimiter", ";").option("header", "true").csv('/datasets/project/istdaten/*/*')

In [8]:
# Keep only the needed information
df = df.select(
          df['BETRIEBSTAG'].alias('date'), 
          df['FAHRT_BEZEICHNER'].alias('id'), 
          df['PRODUKT_ID'].alias('transport_type'), 
          df['LINIEN_ID'].alias('train_number'), 
          df['HALTESTELLEN_NAME'].alias('stop_name'), 
          df['ANKUNFTSZEIT'].alias('arrival_time'), 
          df['AN_PROGNOSE'].alias('actual_arrival_time'),
          df['AN_PROGNOSE_STATUS'].alias('status_arrival_time'),
          df['ABFAHRTSZEIT'].alias('departure_time'),
          df['AB_PROGNOSE'].alias('actual_departure_time'),
          df['AB_PROGNOSE_STATUS'].alias('status_departure_time'),
          df['DURCHFAHRT_TF'].alias('stop_here'))

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pandas import Series
import datetime
import time

formatTS1 = "dd.MM.yyyy HH:mm:ss"
formatTS2 = "dd.MM.yyyy HH:mm"

# compute the departure and arrival delays
departure_delay = (F.unix_timestamp('actual_departure_time', formatTS1) - 
        F.unix_timestamp('departure_time', formatTS2))
arrival_delay = (F.unix_timestamp('actual_arrival_time', formatTS1) - 
        F.unix_timestamp('arrival_time', formatTS2))

# extract the day of the week
def get_week_day(date):
    converted_date = datetime.datetime.strptime(str(date), "%d.%m.%Y")
    return converted_date.weekday()

week_day_udf = F.udf(get_week_day, IntegerType())

# udf to fill NaN departure (arrival) values with the arrival (departure) time
@udf('string')
def fillWithOther(a, b):
    if (a is None):
        return b
    return a
    
# date parser
date = F.to_date('date', 'dd.MM.yyyy')

# First filtering. arrival_delay and departure_delay columns
df_filtered = (df.filter((df.status_arrival_time == 'GESCHAETZT') | (df.status_departure_time == 'GESCHAETZT'))
     .withColumn("arrival_delay", arrival_delay)
     .withColumn("departure_delay", departure_delay)
                      )

# Handling the NaN values
df_with_delays = df_filtered.select('id', 'train_number', 'stop_name', 'arrival_time', 'arrival_delay', 'departure_time', 'departure_delay')
df_with_delays = df_with_delays.fillna(0, ['departure_delay', 'arrival_delay'])
df_with_delays = (df_with_delays
                  .withColumn('arrival_time2', fillWithOther(df_with_delays.arrival_time, df_with_delays.departure_time))
                  .withColumn('departure_time2', fillWithOther(df_with_delays.departure_time, df_with_delays.arrival_time))
                  .drop(df_with_delays.departure_time).drop(df_with_delays.arrival_time)
                 )
df_with_delays = df_with_delays.withColumnRenamed('departure_time2', 'departure_time').withColumnRenamed('arrival_time2', 'arrival_time')
# Adding the day of the week
df_with_delays = df_with_delays.withColumn("day_week", week_day_udf(F.split(df_with_delays.arrival_time, " ")[0]))

# Filtering outliers
df_with_delays = df_with_delays.filter((df_with_delays['arrival_delay'] > -1000) &
                                       (df_with_delays['arrival_delay'] < 1000) &
                                       (df_with_delays['departure_delay'] > -1000) &
                                       (df_with_delays['departure_delay'] < 1000)
                                      )

In [10]:
#df_with_delays.take(100)

In [11]:
# Filtering the stops that are not in the metadata
# Adding a new column 'info' containing all the needed information
# Grouping by trip id
# Collect the list of the 'info' rows in the group
df_grouped = (df_with_delays
    .filter(df_with_delays.stop_name.isin(metadataPandas.stop.tolist()))
    .withColumn('info', F.struct('stop_name', F.split('arrival_time', " ")[1], 'arrival_delay', F.split('departure_time', " ")[1], 'departure_delay', 'day_week'))
    .groupby('id')
    .agg(F.collect_list('info').alias('info'))
              )

In [12]:
# Collecting the groups
groups = df_grouped.collect()

In [21]:
import pandas as pd
import numpy as np
import time
import datetime

In [35]:
# Function to compute the delta between departur and arrival time. If the delta is too high, it will be filtered
def compute_delta(arrival, departure, day):
    d1 = datetime.datetime.strptime(arrival, "%H:%M")
    d2 = datetime.datetime.strptime(departure, "%H:%M")
    d1_ts = time.mktime(d1.timetuple())
    d2_ts = time.mktime(d2.timetuple())
    res = int(d2_ts-d1_ts) / 60
    if (day > 0):
        res = res + 60 * 24 * day
    return res

def delays(s):
    return tuple(s)
    if s.shape[0] < 2:
        return s
    return list(s)

# DataFrame containing the final schedule
result = pd.DataFrame(columns= ['trip_id', 'start', 'stop', 
                               'arrival_time_1', 'departure_time_1', 
                               'arrival_time_2', 'departure_time_2',
                               'mean_delay', 'std_delay', 'day_1', 'day_2', 'delta', 'delays'])

# Iterating through groups
for group in groups:
    trip_id = group[0]
    info = group[1]
    p = pd.DataFrame(info, columns=['stop', 'arrival_time', 'arrival_delay','departure_time', 'departure_delay', 'week_day'])
    
    # Computing the mean of arrival and departure delays
    c = p.groupby(['week_day', 'arrival_time', 'departure_time', 'stop']).agg([np.mean, np.std, delays])

    c = c.reset_index()
    c.columns = [' '.join(col).strip() for col in c.columns.values]
    
    # Creating all the couples of (start_station, arrival_station) for each trip. The order of the schedule 
    # is manteined by the groupby operation. We can have the same schedule for different days of the week, 
    # or the schedule can even change on different days. The compute_delta function calculates the time interval
    # between each couple: if the interval is too high, it means that a new trip started and the couple has to be
    # discarded. We also consider trips that are scheduled between midnight (in this case the week_day changes). 
    partialResult = pd.DataFrame([(trip_id,
                        c.loc[i, 'stop'], 
                        c.loc[i+1, 'stop'], 
                        c.loc[i, 'arrival_time'],
                        c.loc[i, 'departure_time'], 
                        c.loc[i+1, 'arrival_time'], 
                        c.loc[i+1, 'departure_time'], 
                        c.loc[i+1, 'arrival_delay mean'],
                        c.loc[i+1, 'arrival_delay std'], 
                        c.loc[i, 'week_day'], 
                        c.loc[i+1, 'week_day'], 
                        compute_delta(c.loc[i, 'departure_time'], 
                                      c.loc[i+1, 'arrival_time'], 
                                      c.loc[i+1, 'week_day']-c.loc[i, 'week_day']),
                        c.loc[i+1, 'arrival_delay delays']) 
                                  for i in range(len(c)-1)],
                     columns= ['trip_id', 'start', 'stop', 
                               'arrival_time_1', 'departure_time_1', 
                               'arrival_time_2', 'departure_time_2',
                               'mean_delay', 'std_delay', 'day_1', 'day_2', 'delta', 'delays'])
    # Need to merge the last station with the first, in case of a trip that starts on sunday night and finisches
    # on monday morning
    m = len(c)-1
    partialResult = partialResult.append([{'trip_id': trip_id,
                'start': c.loc[m, 'stop'], 
                'stop': c.loc[0, 'stop'], 
                'arrival_time_1': c.loc[m, 'arrival_time'],
                'departure_time_1': c.loc[m, 'departure_time'], 
                'arrival_time_2': c.loc[0, 'arrival_time'],
                'departure_time_2': c.loc[0, 'departure_time'],
                'mean_delay': c.loc[m, 'arrival_delay mean'], 
                'std_delay': c.loc[m, 'arrival_delay std'], 
                'day_1': c.loc[m, 'week_day'], 
                'day_2': c.loc[0, 'week_day'],
                'delta': compute_delta(c.loc[m, 'departure_time'], c.loc[0, 'arrival_time'], c.loc[0, 'week_day']+7-c.loc[m, 'week_day']),
                'delays': c.loc[m, 'arrival_delay delays']
                 }], ignore_index=True
               )
    # Filtering the wrong couples and couples where the start and stop stations are the same
    partialResult = partialResult[partialResult.delta < 180]
    partialResult = partialResult[partialResult.start != partialResult.stop]
    #partialResult = partialResult.drop(['delta'], axis=1)
    result = result.append(partialResult)

result.head()

,trip_id,start,stop,arrival_time_1,departure_time_1,arrival_time_2,departure_time_2,mean_delay,std_delay,day_1,day_2,delta,delays
1,85:11:18388:001,Dietlikon,Stettbach,23:13,23:14,23:17,23:18,60.151515,105.014023,0,0,3.0,"(-2, 62, 257, 194, 233, 75, -20, -8, -5, 34, 4..."
2,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:17,23:18,23:22,23:23,67.666667,97.935459,0,0,4.0,"(1, 13, 219, 187, 203, 44, 13, 1, 12, 148, 14,..."
3,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:22,23:23,23:26,23:29,14.606061,92.191695,0,0,3.0,"(-45, -46, 135, 125, 143, -9, -17, -44, -44, 1..."
4,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:26,23:29,23:31,23:31,40.606061,38.486150,0,0,2.0,"(38, 36, 15, 47, 30, 52, 23, 17, 17, 36, 20, 3..."
5,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31,23:31,23:35,23:36,3.545455,40.070322,0,0,4.0,"(-22, -17, 2, 18, -21, 26, -15, -37, -23, -21,..."


In [36]:
result[result['std_delay'].isnull()]

,trip_id,start,stop,arrival_time_1,departure_time_1,arrival_time_2,departure_time_2,mean_delay,std_delay,day_1,day_2,delta,delays
5,85:11:13752:001,Zürich Stadelhofen,Zürich HB,01:52,01:52,01:55,01:57,-43.0,NaN,4,4,3.0,"(-43,)"
6,85:11:13752:001,Zürich HB,Zürich Hardbrücke,01:55,01:57,01:59,01:59,18.0,NaN,4,4,2.0,"(18,)"
7,85:11:13752:001,Zürich Hardbrücke,Zürich Altstetten,01:59,01:59,02:01,02:01,95.0,NaN,4,4,2.0,"(95,)"
8,85:11:13752:001,Zürich Altstetten,Bonstetten-Wettswil,02:01,02:01,02:16,02:16,24.0,NaN,4,4,15.0,"(24,)"
0,85:11:89922:002,Zürich HB,Zürich Altstetten,06:41,06:41,06:46,06:47,210.0,NaN,4,4,5.0,"(210,)"
0,85:11:88658:003,Zürich HB,Zürich Hardbrücke,15:31,15:31,15:33,15:33,168.0,NaN,2,2,2.0,"(168,)"
1,85:11:88658:003,Zürich Hardbrücke,Zürich Oerlikon,15:33,15:33,15:38,15:39,148.0,NaN,2,2,5.0,"(148,)"
2,85:11:88658:003,Zürich Oerlikon,Zürich Seebach,15:38,15:39,15:41,15:41,152.0,NaN,2,2,2.0,"(152,)"
3,85:11:88658:003,Zürich Seebach,Zürich Affoltern,15:41,15:41,15:44,15:44,162.0,NaN,2,2,3.0,"(162,)"
4,85:11:88658:003,Zürich Affoltern,Regensdorf-Watt,15:44,15:44,15:48,15:49,129.0,NaN,2,2,4.0,"(129,)"


In [37]:
# We found some wrong schedules, always between Zurich HB and Zurich Oerlikon. 
# We suppose that these are errors, because in other days the same schedule doesn't include Zurich Oerlikon
result[(result['departure_time_1'] > result['arrival_time_2']) & (result['day_1'] == result['day_2'])].head()

,trip_id,start,stop,arrival_time_1,departure_time_1,arrival_time_2,departure_time_2,mean_delay,std_delay,day_1,day_2,delta,delays
2,85:11:19256:001,Zürich HB,Zürich Oerlikon,15:11,15:14,15:12,15:12,-66.0,NaN,0,0,-2.0,"(-66,)"
2,85:11:19254:001,Zürich HB,Zürich Oerlikon,14:41,14:44,14:42,14:42,-139.0,NaN,0,0,-2.0,"(-139,)"
2,85:11:19253:001,Zürich HB,Zürich Oerlikon,14:15,14:17,14:16,14:16,54.0,NaN,0,0,-1.0,"(54,)"
17,85:11:19285:001,Zürich HB,Zürich Oerlikon,22:15,22:17,22:16,22:16,156.0,NaN,3,3,-1.0,"(156,)"
2,85:11:19247:001,Zürich HB,Zürich Oerlikon,12:45,12:47,12:46,12:46,94.0,NaN,0,0,-1.0,"(94,)"


In [38]:
# These are the trips that have to be modified
trip_id_to_modify = result[(result['departure_time_1'] > result['arrival_time_2']) & (result['day_1'] == result['day_2'])]['trip_id']

In [39]:
result = result.reset_index().drop(['index'], axis=1)
result.head()

,trip_id,start,stop,arrival_time_1,departure_time_1,arrival_time_2,departure_time_2,mean_delay,std_delay,day_1,day_2,delta,delays
0,85:11:18388:001,Dietlikon,Stettbach,23:13,23:14,23:17,23:18,60.151515,105.014023,0,0,3.0,"(-2, 62, 257, 194, 233, 75, -20, -8, -5, 34, 4..."
1,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:17,23:18,23:22,23:23,67.666667,97.935459,0,0,4.0,"(1, 13, 219, 187, 203, 44, 13, 1, 12, 148, 14,..."
2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:22,23:23,23:26,23:29,14.606061,92.191695,0,0,3.0,"(-45, -46, 135, 125, 143, -9, -17, -44, -44, 1..."
3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:26,23:29,23:31,23:31,40.606061,38.486150,0,0,2.0,"(38, 36, 15, 47, 30, 52, 23, 17, 17, 36, 20, 3..."
4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31,23:31,23:35,23:36,3.545455,40.070322,0,0,4.0,"(-22, -17, 2, 18, -21, 26, -15, -37, -23, -21,..."


In [40]:
# Modifying the wrong rows
for i, row in result.iterrows():
    if ((result.loc[i, 'trip_id'] in list(trip_id_to_modify.values)) &
       (result.loc[i, 'stop'] == 'Zürich Oerlikon')):
        result.loc[i, 'stop'] = result.loc[i+1, 'stop']
        result.loc[i, 'arrival_time_2'] = result.loc[i+1, 'arrival_time_2']
        result.loc[i, 'departure_time_2'] = result.loc[i+1, 'departure_time_2']
        result.loc[i, 'mean_delay'] = result.loc[i+1, 'mean_delay']
        result.loc[i, 'std_delay'] = result.loc[i+1, 'std_delay']

In [41]:
# Eliminating the wrong rows
result = result[ (~result['trip_id'].isin(list(trip_id_to_modify.values)))
        | ((result['trip_id'].isin(list(trip_id_to_modify.values)))
           & (result['start'] != 'Zürich Oerlikon'  ) )]

In [42]:
result

,trip_id,start,stop,arrival_time_1,departure_time_1,arrival_time_2,departure_time_2,mean_delay,std_delay,day_1,day_2,delta,delays
0,85:11:18388:001,Dietlikon,Stettbach,23:13,23:14,23:17,23:18,60.151515,105.014023,0,0,3.0,"(-2, 62, 257, 194, 233, 75, -20, -8, -5, 34, 4..."
1,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:17,23:18,23:22,23:23,67.666667,97.935459,0,0,4.0,"(1, 13, 219, 187, 203, 44, 13, 1, 12, 148, 14,..."
2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:22,23:23,23:26,23:29,14.606061,92.191695,0,0,3.0,"(-45, -46, 135, 125, 143, -9, -17, -44, -44, 1..."
3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:26,23:29,23:31,23:31,40.606061,38.486150,0,0,2.0,"(38, 36, 15, 47, 30, 52, 23, 17, 17, 36, 20, 3..."
4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31,23:31,23:35,23:36,3.545455,40.070322,0,0,4.0,"(-22, -17, 2, 18, -21, 26, -15, -37, -23, -21,..."
5,85:11:18388:001,Dietlikon,Stettbach,23:13,23:14,23:17,23:18,85.812500,133.801354,1,1,3.0,"(-17, 12, 186, 226, 407, 47, 578, 13, 26, 123,..."
6,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:17,23:18,23:22,23:23,82.312500,138.159897,1,1,4.0,"(10, 1, 157, 208, 415, 3, 548, 0, 27, 133, 15,..."
7,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:22,23:23,23:26,23:29,30.218750,130.796008,1,1,3.0,"(-49, -10, 86, 130, 334, -36, 478, -45, -4, 73..."
8,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:26,23:29,23:31,23:31,59.531250,78.816645,1,1,2.0,"(39, 34, 51, 31, 218, 17, 375, 54, 54, 44, 30,..."
9,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31,23:31,23:35,23:36,23.906250,77.567602,1,1,4.0,"(19, 13, 20, -25, 162, -13, 339, 15, 19, -11, ..."


In [44]:
# change time type and drop/rename columns
result2 = (result.drop(['arrival_time_1', 'departure_time_2', 'delta'], axis=1))

def make_time(entry):
    #h, m = entry.split(':')
    return pd.Timedelta(entry+':00')

result2['departure_time_1'] = result2['departure_time_1'].apply(lambda x: make_time(x))
result2['arrival_time_2'] = result2['arrival_time_2'].apply(lambda x: make_time(x))
result2 = result2.rename(columns={'departure_time_1': 'departure_time', 'arrival_time_2': 'arrival_time'})
          
result2.head()

,trip_id,start,stop,departure_time,arrival_time,mean_delay,std_delay,day_1,day_2,delays
0,85:11:18388:001,Dietlikon,Stettbach,23:14:00,23:17:00,60.151515,105.014023,0,0,"(-2, 62, 257, 194, 233, 75, -20, -8, -5, 34, 4..."
1,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:18:00,23:22:00,67.666667,97.935459,0,0,"(1, 13, 219, 187, 203, 44, 13, 1, 12, 148, 14,..."
2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:23:00,23:26:00,14.606061,92.191695,0,0,"(-45, -46, 135, 125, 143, -9, -17, -44, -44, 1..."
3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:29:00,23:31:00,40.606061,38.486150,0,0,"(38, 36, 15, 47, 30, 52, 23, 17, 17, 36, 20, 3..."
4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31:00,23:35:00,3.545455,40.070322,0,0,"(-22, -17, 2, 18, -21, 26, -15, -37, -23, -21,..."


In [46]:
# Compute the walking times between stations. Keeping only stations within 300m
walk = pd.DataFrame(columns= ['trip_id', 'start', 'stop', 
                               'departure_time', 
                               'arrival_time',
                               'mean_delay', 'std_delay', 'day_1', 'day_2', 'delays'])
for index1, stop1 in metadataPandas.iterrows():
    for index2, stop2 in metadataPandas.iterrows():
        if (stop1.stop != stop2.stop):
            distance = compute_distance(float(stop1.latitude), float(stop1.longitude), 
                                       float(stop2.latitude), float(stop2.longitude))
            if (distance <= 300):
                walk = walk.append({'trip_id': '0000', 
                                    'start': stop1.stop, 
                                    'stop': stop2.stop, 
                                    'mean_delay': distance, # considering 1m/s
                                    'std_delay': 0.0}, ignore_index=True)

In [47]:
walk.head()

,trip_id,start,stop,departure_time,arrival_time,mean_delay,std_delay,day_1,day_2,delays
0,0000,Bonstetten-Wettswil,"Bonstetten-Wettswil, Bahnhof",NaN,NaN,57.121084,0.0,NaN,NaN,NaN
1,0000,"Waldegg, Birmensdorferstrasse","Waldegg, Post",NaN,NaN,163.432216,0.0,NaN,NaN,NaN
2,0000,"Waldegg, Birmensdorferstrasse","Uitikon Waldegg, Bahnhof",NaN,NaN,287.582974,0.0,NaN,NaN,NaN
3,0000,"Zürich, Goldbrunnenplatz","Zürich, Zwinglihaus",NaN,NaN,262.946946,0.0,NaN,NaN,NaN
4,0000,Zürich HB,Zürich HB SZU,NaN,NaN,140.199392,0.0,NaN,NaN,NaN


In [48]:
result3 = result2.append(walk)

In [49]:
result3.head()

,trip_id,start,stop,departure_time,arrival_time,mean_delay,std_delay,day_1,day_2,delays
0,85:11:18388:001,Dietlikon,Stettbach,23:14:00,23:17:00,60.151515,105.014023,0,0,"(-2, 62, 257, 194, 233, 75, -20, -8, -5, 34, 4..."
1,85:11:18388:001,Stettbach,Zürich Stadelhofen,23:18:00,23:22:00,67.666667,97.935459,0,0,"(1, 13, 219, 187, 203, 44, 13, 1, 12, 148, 14,..."
2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,23:23:00,23:26:00,14.606061,92.191695,0,0,"(-45, -46, 135, 125, 143, -9, -17, -44, -44, 1..."
3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,23:29:00,23:31:00,40.606061,38.486150,0,0,"(38, 36, 15, 47, 30, 52, 23, 17, 17, 36, 20, 3..."
4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,23:31:00,23:35:00,3.545455,40.070322,0,0,"(-22, -17, 2, 18, -21, 26, -15, -37, -23, -21,..."


In [50]:
result3[result3['std_delay'].isnull()]

,trip_id,start,stop,departure_time,arrival_time,mean_delay,std_delay,day_1,day_2,delays
406,85:11:13752:001,Zürich Stadelhofen,Zürich HB,01:52:00,01:55:00,-43.0,NaN,4,4,"(-43,)"
407,85:11:13752:001,Zürich HB,Zürich Hardbrücke,01:57:00,01:59:00,18.0,NaN,4,4,"(18,)"
408,85:11:13752:001,Zürich Hardbrücke,Zürich Altstetten,01:59:00,02:01:00,95.0,NaN,4,4,"(95,)"
409,85:11:13752:001,Zürich Altstetten,Bonstetten-Wettswil,02:01:00,02:16:00,24.0,NaN,4,4,"(24,)"
438,85:11:89922:002,Zürich HB,Zürich Altstetten,06:41:00,06:46:00,210.0,NaN,4,4,"(210,)"
439,85:11:88658:003,Zürich HB,Zürich Hardbrücke,15:31:00,15:33:00,168.0,NaN,2,2,"(168,)"
440,85:11:88658:003,Zürich Hardbrücke,Zürich Oerlikon,15:33:00,15:38:00,148.0,NaN,2,2,"(148,)"
441,85:11:88658:003,Zürich Oerlikon,Zürich Seebach,15:39:00,15:41:00,152.0,NaN,2,2,"(152,)"
442,85:11:88658:003,Zürich Seebach,Zürich Affoltern,15:41:00,15:44:00,162.0,NaN,2,2,"(162,)"
443,85:11:88658:003,Zürich Affoltern,Regensdorf-Watt,15:44:00,15:48:00,129.0,NaN,2,2,"(129,)"


In [55]:
result3.delays.dropna().apply(lambda x: len(x)).describe()

count    98409.000000
mean        20.071670
std         11.553257
min          1.000000
25%         12.000000
50%         20.000000
75%         32.000000
max         33.000000
Name: delays, dtype: float64

In [51]:
result3[result3['trip_id'] == '85:11:88325:005']

,trip_id,start,stop,departure_time,arrival_time,mean_delay,std_delay,day_1,day_2,delays
98424,85:11:88325:005,Zürich HB,Zürich Stadelhofen,07:03:00,07:05:00,34.0,NaN,5,5,"(34,)"
98425,85:11:88325:005,Zürich Stadelhofen,Stettbach,07:06:00,07:10:00,7.0,NaN,5,5,"(7,)"
98426,85:11:88325:005,Stettbach,Dietlikon,07:11:00,07:13:00,72.0,NaN,5,5,"(72,)"


In [171]:
result3.to_pickle('/home/succa/final_project/schedule.csv')

In [20]:
res = pd.read_pickle('/home/succa/final_project/schedule.csv', parse_dates=True)

In [21]:
res.head()

,Unnamed: 0,trip_id,start,stop,departure_time,arrival_time,mean_delay,std_delay,day_1,day_2
0,0,85:11:18388:001,Dietlikon,Stettbach,0 days 23:14:00.000000000,0 days 23:17:00.000000000,60.151515,105.014023,0.0,0.0
1,1,85:11:18388:001,Stettbach,Zürich Stadelhofen,0 days 23:18:00.000000000,0 days 23:22:00.000000000,67.666667,97.935459,0.0,0.0
2,2,85:11:18388:001,Zürich Stadelhofen,Zürich HB,0 days 23:23:00.000000000,0 days 23:26:00.000000000,14.606061,92.191695,0.0,0.0
3,3,85:11:18388:001,Zürich HB,Zürich Hardbrücke,0 days 23:29:00.000000000,0 days 23:31:00.000000000,40.606061,38.486150,0.0,0.0
4,4,85:11:18388:001,Zürich Hardbrücke,Zürich Altstetten,0 days 23:31:00.000000000,0 days 23:35:00.000000000,3.545455,40.070322,0.0,0.0


In [24]:
res[res['mean_delay'].isnull()]

,Unnamed: 0,trip_id,start,stop,departure_time,arrival_time,mean_delay,std_delay,day_1,day_2
